In [ ]:
pip install memory_profiler

In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from memory_profiler import memory_usage
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier, StackingClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB7
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import psutil
# import resource
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve, auc

In [ ]:
# Define the image size
import os
IMG_SIZE = 50

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Anomaly Detection/CSV


train1_df = pd.read_csv("./train_feature_index.csv")
test1_df = pd.read_csv("./test_feature_index.csv")

train1_df = train1_df.dropna(axis=0, how='all').iloc[:-2]
test1_df = test1_df.dropna(axis=0, how='all').iloc[:-2]

train1_df.to_csv("train_cleaned.csv", index=False)
test1_df.to_csv("test_cleaned.csv", index=False)

train_df = pd.read_csv("train_cleaned.csv")
test_df = pd.read_csv("test_cleaned.csv")

# train_df = pd.read_csv("./train_final_index.csv")
# test_df = pd.read_csv("./test_final_index.csv")
# # train_df = pd.read_csv("./train_feature_index_1.csv")
# # test_df = pd.read_csv("./test_feature_index_1.csv")

# train_df = pd.read_csv("./train_feature_index_2.csv")
# test_df = pd.read_csv("./test_feature_index_2.csv")

path = "/content/drive/MyDrive/Colab Notebooks/Anomaly Detection/fiveTwelve"
print("found", os.path.exists(path=path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Anomaly Detection/CSV
found True


In [ ]:
X_train = train_df.drop(columns=["label", "path"])
y_train = train_df["label"]

X_test = test_df.drop(columns=["label", "path"])
y_test = test_df["label"]

def load_and_preprocess_image(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    return image

In [ ]:
#Remove the memory code after testing
# Sample 1
process = psutil.Process()
before_memory = process.memory_info().rss / 1024 / 1024  # Convert to MB
print("process", process, "before Memory", before_memory)

process psutil.Process(pid=142, name='python3', status='running', started='09:37:55') before Memory 650.28515625


In [ ]:
X_train_reorder = np.random.permutation(X_train)

knn = KNeighborsClassifier()

params_knn = {"n_neighbors": np.arange(1, 25)}

knn_gs = GridSearchCV(knn, params_knn, cv=10)

knn_gs.fit(X_train, y_train)
knn_gs.fit(X_train_reorder, y_train)

best_params = knn_gs.best_params_
best_score = knn_gs.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

used_algorithm = knn.algorithm

knn_best = knn_gs.best_estimator_
print(knn_gs.best_params_)


X_train_reorder = np.random.permutation(X_train)

rf = RandomForestClassifier()
params_rf = {"n_estimators": [50, 100, 200]}
rf_gs = GridSearchCV(rf, params_rf, cv=10)
rf_gs.fit(X_train, y_train)
rf_gs.fit(X_train_reorder, y_train)


rf_best = rf_gs.best_estimator_
print(rf_gs.best_params_)
rf_best_params = rf_gs.best_params_
rf_best_score = rf_gs.best_score_

print("Best Parameters:", rf_best_params)
print("Best Score:", rf_best_score)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

print("knn: {}".format(knn_best.score(X_test, y_test)))
print("rf: {}".format(rf_best.score(X_test, y_test)))


estimators=[("knn", knn_best), ("rf", rf_best)]
ensemble = VotingClassifier(estimators, voting="hard")
# ensemble = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

ensemble.fit(X_train, y_train)

ens_best = ensemble.score(X_test, y_test)

print("Ens_best", ens_best)

y_pred = ensemble.predict(X_test)

y_pred_labels = y_pred

ens_labels = ensemble.predict(X_test)

score1 = knn_best.score(X_test, y_test)
score2 = rf_best.score(X_test, y_test)

Best Parameters: {'n_neighbors': 8}
Best Score: 0.9788859748803633
{'n_neighbors': 8}
{'n_estimators': 100}
Best Parameters: {'n_estimators': 100}
Best Score: 0.9788859748803633
X_train shape: (46462, 11)
X_test shape: (46462, 11)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


knn: 0.9788859713314106


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


rf: 0.9586759071929749
Ens_best 0.9845680340923766


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [ ]:
def image_generator(X, y, batch_size=16):
    num_samples = len(X)
    indices = np.arange(num_samples)
    while True:
        np.random.shuffle(indices)
        batch_indices = []
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            batch_indices.extend(indices[start:end])

            if len(batch_indices) == batch_size:
                batch_X = np.zeros((batch_size, 224, 224, 3))
                batch_y = np.zeros((batch_size,))  # Use num_classes from your problem

                for i, idx in enumerate(batch_indices):
                    if idx >= num_samples:
                        continue  # Skip this sample

                    if "path" not in X.iloc[idx]:
                        continue  # Skip this sample if 'path' key is missing

                    path = X.iloc[idx]["path"]
                    image = load_and_preprocess_image(path)
                    if image is None:
                        continue  # Skip this sample

                    batch_X[i] = image
                    batch_y[i] = y.iloc[idx]

                yield batch_X, batch_y
                batch_indices = []

        # If there are remaining samples that don't form a full batch, process them as well
        if batch_indices:
            batch_X = np.zeros((len(batch_indices), 224, 224, 3))
            batch_y = np.zeros((len(batch_indices), 10))  # Use num_classes from your problem
            for i, idx in enumerate(batch_indices):
                if idx >= num_samples:
                    continue  # Skip this sample

                if "path" not in X.iloc[idx]:
                    continue  # Skip this sample if 'path' key is missing

                path = X.iloc[idx]["path"]
                image = load_and_preprocess_image(path)
                if image is None:
                    continue  # Skip this sample

                batch_X[i] = image
                batch_y[i] = y.iloc[idx]

            yield batch_X, batch_y

# def image_generator(X, y, batch_size=32):
#     num_samples = len(X)
#     indices = np.arange(num_samples)
#     while True:
#         np.random.shuffle(indices)  # Shuffle the indices at the beginning of each epoch
#         batch_indices = []
#         for start in range(0, num_samples, batch_size):
#             end = min(start + batch_size, num_samples)
#             batch_indices.extend(indices[start:end])

#             if len(batch_indices) == batch_size:
#                 batch_X = np.zeros((batch_size, 224, 224, 3))
#                 batch_y = np.zeros((batch_size,))
#                 for i, idx in enumerate(batch_indices):
#                     path = X.iloc[idx]["path"]
#                     image = load_and_preprocess_image(path)
#                     label = y.iloc[idx]
#                     X_modified = X.drop('path', axis=1)
#                     X_modified['path_id'] = range(1, len(X_modified) + 1)

#                     input_data = pd.DataFrame([X_modified.iloc[idx]])
#                     prediction = ensemble.predict(input_data)

#                     batch_X[i] = image
#                     batch_y[i] = prediction
#                 yield batch_X, batch_y
#                 batch_indices = []  # Reset batch_indices for the next batch

#         # If there are remaining samples that don't form a full batch, process them as well
#         if batch_indices:
#             batch_X = np.zeros((len(batch_indices), 224, 224, 3))
#             batch_y = np.zeros((len(batch_indices),))
#             for i, idx in enumerate(batch_indices):
#                 path = X.iloc[idx]["path"]
#                 image = load_and_preprocess_image(path)
#                 label = y.iloc[idx]
#                 X_modified = X.drop('path', axis=1)
#                 X_modified['path_id'] = range(1, len(X_modified) + 1)

#                 input_data = pd.DataFrame([X_modified.iloc[idx]])
#                 prediction = ensemble.predict(input_data)

#                 batch_X[i] = image
#                 batch_y[i] = prediction
#             yield batch_X, batch_y


# def load_and_preprocess_image(path):
#     image = cv2.imread(path)
#     if image is not None:
#       image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#       image = cv2.resize(image, (224, 224))
#       image = image / 255.0
#     return image


X_train = train_df.drop(columns=["label", "path_id"])
y_train = train_df["label"]

X_test = test_df.drop(columns=["label", "path_id"])
y_test = test_df["label"]


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import matplotlib.pyplot as plt
import numpy as np

def run_code():

    num_classes = 10
    base_model = EfficientNetB7(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
    base_model.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(10, activation="softmax")
    ])

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    # Assuming X_train, y_train, X_test, y_test are your data arrays
    batch_size = 4  # Adjust as needed

    num_classes = 10
    # One-hot encode the labels
    y_train_encoded = to_categorical(y_train, num_classes=num_classes)
    y_test_encoded = to_categorical(y_test, num_classes=num_classes)

    # Creating data generators
    train_generator = image_generator(X_train, y_train, batch_size)
    test_generator = image_generator(X_test, y_test, batch_size)

    epochs = 10
    accuracy_history = []
    loss_history = []


    epochs = 10
    accuracy_history = []
    loss_history = []

    for epoch in range(epochs):
        # Using the generator to provide batches of data
        history = model.fit(train_generator, epochs=epoch+1, validation_data=test_generator)

        # Append the accuracy and loss lists for all epochs to history
        accuracy_history.append(history.history['accuracy'][0])
        loss_history.append(history.history['loss'][0])

    print("Completed")

    # Evaluate the model on the test data
    test_loss, test_acc = model.evaluate(test_generator)
    print("Test Loss:", test_loss, "Test accuracy:", test_acc)

    # Assuming test_generator provides batches of data
    y_pred = model.predict(test_generator)
    y_pred_labels = np.argmax(y_pred, axis=1)

    report = classification_report(y_test, y_pred_labels)
    print(report)

    cm = confusion_matrix(y_test, y_pred_labels)
    print("Confusion Matrix:")
    print(cm)

    f1 = f1_score(y_test, y_pred_labels, average='macro')
    print("F1 Score:", f1)

    precision, recall, thresholds = precision_recall_curve(y_test_processed, y_pred[:, 1])

    auc_pr = auc(recall, precision)

    plt.figure()
    plt.plot(recall, precision, color='blue', lw=2, label='Precision-Recall curve (AUC = %0.2f)' % auc_pr)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc='lower left')
    plt.show()

    #plot accuracy
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, epochs + 1), accuracy_history, marker='o', linestyle='-')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.grid(True)
    plt.xticks(np.arange(1, epochs + 1))
    plt.ylim(0, 1)
    plt.show()

    #plot loss
    plt.plot(range(1, epochs + 1), loss_history, marker='o', linestyle='-')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.xticks(np.arange(1, epochs + 1))
    plt.ylim(0, 1)
    plt.show()

    return 1



In [ ]:
run_code()

after_memory = process.memory_info().rss / 1024 / 1024  # Convert to MB
memory_usage = after_memory - before_memory

print("Memory usage:", memory_usage, "MB")

    653/Unknown - 5380s 8s/step - loss: 14.3723 - accuracy: 0.4118

In [ ]:
def image_generator(X, y, batch_size=32):
    num_samples = len(X)
    indices = np.arange(num_samples)
    while True:
        np.random.shuffle(indices)  # Shuffle the indices at the beginning of each epoch
        batch_indices = []
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            batch_indices.extend(indices[start:end])

            if len(batch_indices) == batch_size:
                batch_X = np.zeros((batch_size, 224, 224, 3))
                batch_y = np.zeros((batch_size,))
                for i, idx in enumerate(batch_indices):
                    path = X.iloc[idx]["path"]
                    image = load_and_preprocess_image(path)
                    label = y.iloc[idx]
                    X_modified = X.drop('path', axis=1)
                    X_modified['path_id'] = range(1, len(X_modified) + 1)

                    input_data = pd.DataFrame([X_modified.iloc[idx]])
                    prediction = ensemble.predict(input_data)

                    batch_X[i] = image
                    batch_y[i] = prediction
                yield batch_X, batch_y
                batch_indices = []  # Reset batch_indices for the next batch

        # If there are remaining samples that don't form a full batch, process them as well
        if batch_indices:
            batch_X = np.zeros((len(batch_indices), 224, 224, 3))
            batch_y = np.zeros((len(batch_indices),))
            for i, idx in enumerate(batch_indices):
                path = X.iloc[idx]["path"]
                image = load_and_preprocess_image(path)
                label = y.iloc[idx]
                X_modified = X.drop('path', axis=1)
                X_modified['path_id'] = range(1, len(X_modified) + 1)

                input_data = pd.DataFrame([X_modified.iloc[idx]])
                prediction = ensemble.predict(input_data)

                batch_X[i] = image
                batch_y[i] = prediction
            yield batch_X, batch_y



def load_and_preprocess_image(path):
    image = cv2.imread(path)
    if image is not None:
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      image = cv2.resize(image, (224, 224))
      image = image / 255.0
    return image

X_train = train_df.drop(columns=["label", "path_id"])
y_train = train_df["label"]

X_test = test_df.drop(columns=["label", "path_id"])
y_test = test_df["label"]

In [ ]:
def run_code():
    base_model = EfficientNetB7(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
    base_model.trainable = False

    inputs = keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    outputs = keras.layers.Dense(10, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    print("X_test", X_test, "y_test", y_test)
    train_generator = image_generator(X_train, y_train, 500)
    test_generator = image_generator(X_test, y_test, 500)
    print("test_generator",test_generator)

    X_train_processed, y_train_processed = next(train_generator)
    X_test_processed, y_test_processed = next(test_generator)
    print("X_test_processed", len(X_test_processed), "y_test_processed", len(y_test_processed))

    epochs = 5
    accuracy_history = []
    loss_history = []

    for epoch in range(epochs):
        history = model.fit(X_train_processed, y_train_processed, epochs=epoch+1, validation_data=(X_test_processed, y_test_processed))
        batch_size = 2
        # history = model.fit(train_generator, steps_per_epoch=len(X_train_processed)//batch_size, epochs=epochs, validation_data=test_generator, validation_steps=len(X_test_processed)//batch_size)
        # history = model.fit_generator(train_generator, steps_per_epoch=len(X_train) // batch_size, epochs=epochs, validation_data=test_generator, validation_steps=len(X_test) // batch_size)
        # Access training accuracy and loss for all epochs
        training_accuracy = history.history['accuracy'][0]
        training_loss = history.history['loss'][0]

        # Append the accuracy and loss lists for all epochs to history
        accuracy_history.append(training_accuracy)
        loss_history.append(training_loss)

    # model.fit(X_train_processed, y_train_processed, validation_data=(X_test_processed, y_test_processed), epochs=10)

    print("Completed")
    test_loss, test_acc = model.evaluate(X_test_processed, y_test_processed)
    print("Test Loss:",test_loss, "Test accuracy:", test_acc)
    # Check the shape and data type of y_train_processed and y_test_processed

    # Assuming y_test_processed contains binary labels (0 and 1)
    y_pred = model.predict(X_test_processed)
    y_pred_labels = y_pred.argmax(axis=1) # Convert probabilities to binary labels

    # print("y_test_processed", y_test_processed , "y_pred_labels", y_pred_labels, "X_test_processed", X_test_processed)

    # Generate classification report
    report = classification_report(y_test_processed, y_pred_labels)
    print(report)
    # Convert the data type of y_test_processed to integer
    y_test_processed = y_test_processed.astype(int)

    # Get predicted labels from the model
    y_pred = model.predict(X_test_processed)
    # print("y_pred", y_pred, y_test_processed)
    y_pred_labels = y_pred.argmax(axis=1)

    # Generate the confusion matrix
    cm = confusion_matrix(y_test_processed, y_pred_labels)

    # Print the confusion matrix
    print("Confusion Matrix:")
    print(cm)
    print("confusion_matrix", cm)
    # Calculate the F1 score
    f1 = f1_score(y_test_processed, y_pred_labels, average='macro')
    print("F1 Score:", f1)

    #plot accuracy
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, epochs + 1), accuracy_history, marker='o', linestyle='-')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.grid(True)
    plt.xticks(np.arange(1, epochs + 1))
    plt.ylim(0, 1)
    plt.show()

    #plot loss
    plt.plot(range(1, epochs + 1), loss_history, marker='o', linestyle='-')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.xticks(np.arange(1, epochs + 1))
    plt.ylim(0, 1)
    plt.show()

    precision, recall, thresholds = precision_recall_curve(y_test_processed, y_pred[:, 1])

    # Calculate area under the precision-recall curve (AUC-PR)
    auc_pr = auc(recall, precision)

    # Plot precision-recall curve
    plt.figure()
    plt.plot(recall, precision, color='blue', lw=2, label='Precision-Recall curve (AUC = %0.2f)' % auc_pr)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc='lower left')
    plt.show()
    return 1